# Anomaly calculations relative to a 21-year running mean
The mean state of ocean circulation is changing with climate change. Anomalies are calculated with respect to the ensemble mean of a 21-year running mean for each 5-day mean period during the year.

In [ ]:
import xarray as xr
import numpy as np
from cdo import *
cdo=Cdo()
import datetime as dt
from joblib import Parallel, delayed

Calculate the running mean

In [ ]:
inds=[]
temp=xr.open_dataset('c6_ssh-vo_mon_lin_eq_5day_node_r1_1850-2100_nol.nc')
for j in np.arange(len(temp.time))[730:-730]:
    inds.append(np.arange((j-(10*73)),(j+(11*73)),73))

In [ ]:
def run_mean(i,data):
    return np.mean(data[inds[i],:].values,axis=0) 

In [ ]:
for k in [1,2,3,4,5]:
    #vo
    temp=xr.open_dataset('c6_ssh-vo_mon_lin_eq_5day_node_r'+str(k)+'_1850-2100_nol.nc')
    dtemp=temp.vo
    
    rmvals=Parallel(n_jobs=-1,batch_size=20,verbose=10)(delayed(run_mean)(i,dtemp) for i in np.arange(len(inds)))
    rmvals=np.asarray(rmvals)
    
    tempds=xr.DataArray(data=rmvals,coords={'time':temp.time[730:-730]},
                          dims=['time','ncells'],
                          attrs={'description':'21-year running mean of 5-day mean vo values',
                                 'units':'m/s',
                                })
    tempda=tempds.to_dataset(name='vo')
    tempda=tempda.assign({'time_bnds':temp.time_bnds[730:-730]})
    tempda.to_netcdf('c6_ssh-vo_mon_lin_eq_5day_node_r'+str(k)+'_1860-2090_21yr_seas_rm.nc') 

In [ ]:
for run in [1,2,3,4,5]:
    #uo
    temp=xr.open_dataset('c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1850-2100_nol.nc')
    dtemp=temp.uo
    
    rmvals=Parallel(n_jobs=-1,batch_size=20,verbose=10)(delayed(run_mean)(i,dtemp) for i in np.arange(len(inds)))
    rmvals=np.asarray(rmvals)
    
    tempds=xr.DataArray(data=rmvals,coords={'time':temp.time[730:-730]},
                          dims=['time','ncells'],
                          attrs={'description':'21-year running mean of 5-day mean uo values',
                                 'units':'m/s',
                                })
    tempda=tempds.to_dataset(name='uo')
    tempda=tempda.assign({'time_bnds':temp.time_bnds[730:-730]})
    tempda.to_netcdf('c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_21yr_seas_rm.nc') 

Take advantage of the five-member ensemble to to increase the reference data

In [ ]:
files=[]
for run in [1,2,3,4,5]:
    files.append('c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_21yr_seas_rm.nc')
outp='c6_ssh-uo_mon_lin_eq_5day_node_ensmean_1860-2090_21yr_seas_rm.nc'
cdo.ensmean(input=(files),output=outp)

In [ ]:
files=[]
for run in [1,2,3,4,5]:
    files.append('c6_ssh-vo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_21yr_seas_rm.nc')
outp='c6_ssh-vo_mon_lin_eq_5day_node_ensmean_1860-2090_21yr_seas_rm.nc'
cdo.ensmean(input=(files),output=outp)

Calculate the anomalies relative to the running mean

In [ ]:
#remove time lost to running mean (10years each end)
for run in [1,2,3,4,5]:
    temp=xr.open_dataset('c6_ssh-vo_mon_lin_eq_5day_node_r'+str(run)+'_1850-2100_nol.nc')
    temp=temp.sel({'time':temp.time[730:-730]})
    outp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_nol.nc'
    temp.to_netcdf(outp)

In [ ]:
for run in [1,2,3,4,5]:
    temp=xr.open_dataset('c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1850-2100_nol.nc')
    temp=temp.sel({'time':temp.time[730:-730]})
    outp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_nol.nc'
    temp.to_netcdf(outp)

In [ ]:
#subtract the ensemble mean of running means
mean='c6_ssh-uo_mon_lin_eq_5day_node_ensmean_1860-2090_nol_21yr_seas_rm.nc'

for run in [1,2,3,4,5]:
    inp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_nol.nc'
    outp='c6_ssh-uo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_nol_anom.nc'
    cdo.sub(input=([inp,mean]),output=outp)

In [ ]:
mean='c6_ssh-vo_mon_lin_eq_5day_node_ensmean_1860-2090_nol_21yr_seas_rm.nc'

for run in [1,2,3,4,5]:
    inp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_nol.nc'
    outp='c6_ssh-vo_mon_lin_eq_5day_node_r'+str(run)+'_1860-2090_nol_anom.nc'
    cdo.sub(input=([inp,mean]),output=outp)